In [ ]:
import pandas as pd
import re
import requests
import xml.etree.ElementTree as ET
import time
import os
import csv

In [ ]:
## !!! WE ARE NOT USING THIS !!!
def blind_extraction(text):
  for line in text:
    if line == "\n":
      newLine = True
      count +=1
      if count ==2:
          count = 0
          print(list_1)
          extracted_abstracts.append({
              'PMID': list_1[5],
              'doi': list_1[4],
              'title': list_1[1],
              'authors': list_1[2],
              'text': list_1[3],
          })
          list_1 = []
    else:
        lineData = line
        skip_append_flag = False

        # Extract DOI as a seperate item
        if(lineData.startswith("DOI:")):
          lineData = lineData.replace("DOI:", "")

        #Extract PMID as a seperate item
        if(lineData.startswith("PMID:")):
          lineData = lineData.replace("PMID:", "")
          lineData = lineData.replace("[Indexed for MEDLINE]", "")
          list_1.append(lineData.rstrip().lstrip())
          skip_append_flag = True

        if(lineData.startswith("Copyright")):
          skip_append_flag = True

        if(lineData.startswith("Author information:")):
          skip_append_flag = True
        # Extract publication name --- future work
        # Extract date published --- future work

        if(not skip_append_flag):
          if(newLine==False):
            lineData = list_1[-1] +" "+ lineData.rstrip().lstrip()
            list_1[-1] = lineData
          else:
            list_1.append(lineData.rstrip().lstrip())

        count = 0
        newLine=False

In [ ]:
abstract_data_dataframe = pd.DataFrame(columns=['PMID','doi', 'title', 'authors', 'text'])

In [ ]:
def checkCSVFile(file_name):
  if not os.path.isfile(file_name):
    # If the file doesn't exist, create it
    with open(file_name, 'w', newline='') as csvfile:
        # Create a CSV writer object
        csv_writer = csv.writer(csvfile)

    print(f"CSV file '{file_name}' has been created.")
  else:
      print(f"CSV file '{file_name}' already exists.")

In [ ]:
def writeFailedPmid(pmid):
  failed_pmid_filename = "./failedPmids.csv"
  #writing to csv file
  with open(filename, 'a') as csvfile:
    #creating a csv writer object
    csvwriter = csv.writer(csvfile)

    # writing the data rows
    csvwriter.writerows([[pmid]])

In [ ]:
def writeDataToCSVFile(data, abstract_filename):
  # name of csv file
  filename = abstract_filename
  #writing to csv file
  with open(filename, 'a') as csvfile:
    #creating a csv writer object
    csvwriter = csv.writer(csvfile)

    # writing the data rows
    csvwriter.writerows([[data["pmid"], data["doi"], data["authors"], data["title"], data["text"]]])



In [ ]:
def recoverDataFromXML(XML_data):
  extracted_data = {
      "pmid":"",
      "doi":"",
      "authors":"",
      "title":"",
      "text":""
  }
  tree = ET.ElementTree(ET.fromstring(XML_data.replace('\n', '')))
  for elem in tree.iter():
    if(elem.tag == "ArticleTitle"):
      extracted_data["title"] = elem.text
    if(elem.tag == "ArticleId"):
      if(elem.attrib["IdType"] == "doi"):
        extracted_data["doi"] = elem.text
    if(elem.tag == "AbstractText"):
      extracted_data["text"] = elem.text
    if(elem.tag == "AuthorList"):
      author_list = []
      for child in elem:
        if(child.find("ForeName") is not None and child.find("LastName") is not None):
          author_name=str(getattr(child.find("ForeName"),"text"))+" "+str(getattr(child.find("LastName"),"text"))
          author_list.append(author_name)
      extracted_data["authors"] = author_list

  return extracted_data

In [ ]:
def getAbstractData(pmID,filename, testResponseMode = False, count = 0) :
  # api-endpoint
  URL = f"https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&id={pmID}&retmode=XML&rettype=abstract&api_key=eb7ce0fc2436e350ccef78a21c6b1a494508"

  # sending get request and saving the response as response object
  r = requests.get(url = URL)
  print(r)
  extractedText = None
  # only for debug mode
  if(testResponseMode):
    print(r.text)
  # write data to CSV when API response is 200
  elif (testResponseMode==False and r.status_code == 200):
    extractedText =  recoverDataFromXML(r.text)
    extractedText["pmid"] = pmID
    writeDataToCSVFile(extractedText, filename)
  #if API request failed (mostly failes due to excess load, wait a second and retry until count hits 5)
  elif (testResponseMode==False and r.status_code != 200 and count<=5):
    time.sleep(1)
    count = count + 1
    getAbstractData(pmID,filename,False,count)

  return True

In [ ]:
getAbstractData(35801624, True)

In [ ]:
def abstract_data_main_function(i):
  print(i)
  logFile = open("./extraction_log.txt","w")
  dataFile = open("./completionRecord.txt","a")
  dataFile.write(f"extracted_text_{i}.csv ----- STARTED \n")
  csv_abstract_filename = f"/content/drive/MyDrive/NLPT(Abstract Data)/extracted_csv_data/extracted_text_{i}.csv"
  checkCSVFile(csv_abstract_filename)
  file = open(f"raw_pmid_data/abstract-intelligen-set ({i}).txt", 'r', encoding="utf-8")
  logFile.write(f" working on file abstracts-original/abstract-intelligen-set ({i}).txt \n")
  print(f"abstracts-original/abstract-intelligen-set ({i}).txt")
  text = file.readlines()
  count = 0
  pmid_extracted_counter = 0
  api_rate_limiter = 0
  pmid_val = ""
  chunk_abstracts = []
  newLine = True
  for line in text:
    if line == "\n":
      newLine = True
      count +=1
      if count ==2:
        count = 0
        if(pmid_val != ""):
          pmid_extracted_counter += 1
          if(pmid_extracted_counter >= 0):
            print(pmid_extracted_counter)
            api_rate_limiter += 1
            logFile.write(f" pmid_extracted_counter: ({pmid_extracted_counter}) \n")
            getAbstractData(pmid_val,csv_abstract_filename,False)
            print("RATE LIMITER: "+ str(api_rate_limiter))
            if(api_rate_limiter >= 9):
                print("RATE LIMIT")
                time.sleep(1)
                api_rate_limiter = 0
          #abstract_data_dataframe = pd.concat([abstract_data_dataframe, pd.DataFrame(extracted_text)], axis=0, ignore_index=True)
        list_1 = []
    else:
      count = 0
      lineData = line
      #Extract PMID as a seperate item
      if(lineData.startswith("PMID:")):
        lineData = lineData.replace("PMID:", "")
        lineData = lineData.replace("[Indexed for MEDLINE]", "")
        pmid_val = lineData.rstrip().lstrip()
  dataFile.write(f"extracted_text_{i}.csv ----- COMLPLETED \n")
  return True

In [ ]:
len(chunk_abstracts)

In [ ]:
def getPMIDs (i):
  chunk_abstracts = []
  file = open(f"/content/drive/MyDrive/NLPT(Abstract Data)/abstracts-original/abstract-intelligen-set ({i}).txt", 'r', encoding="utf-8")
  text = file.readlines()
  count = 0
  pmid_val = ""
  chunk_abstracts = []
  newLine = True
  for line in text:
    if line == "\n":
      newLine = True
      count +=1
      if count ==2:
        count = 0
        if(pmid_val != ""):
          chunk_abstracts.append(pmid_val)
          #abstract_data_dataframe = pd.concat([abstract_data_dataframe, pd.DataFrame(extracted_text)], axis=0, ignore_index=True)
        list_1 = []
    else:
      count = 0
      lineData = line
      #Extract PMID as a seperate item
      if(lineData.startswith("PMID:")):
        lineData = lineData.replace("PMID:", "")
        lineData = lineData.replace("[Indexed for MEDLINE]", "")
        pmid_val = lineData.rstrip().lstrip()
  return chunk_abstracts







In [ ]:
chunk_abstracts = getPMIDs()

In [ ]:
len(chunk_abstracts)

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/NLPT(Abstract Data)/extracted_csv_data/extracted_text_8.csv')

In [ ]:
df.columns = ['PMID','doi', 'title', 'authors', 'text']

In [ ]:
df.head()

In [ ]:
read_pmids = df["PMID"].astype(str).tolist()

In [ ]:
read_pmids[1]

In [ ]:
def find_missing_elements(i):
    arr1 = getPMIDs(i)
    if(not os.path.exists(f'/content/drive/MyDrive/NLPT(Abstract Data)/data_backup/extracted_data/extracted_text_{i}.csv')):
      print(f"{i} : File missing: {len(arr1)}")
      return []

    try:
        df = pd.read_csv(f'/content/drive/MyDrive/NLPT(Abstract Data)/data_backup/extracted_data/extracted_text_{i}.csv', on_bad_lines='skip')
    except pd.errors.EmptyDataError:
        print(f"{i} : File empty: {len(arr1)}")
        return []

    if(df.empty):
      print(f"{i} : File empty: {len(arr1)}")
      return []

    df.columns = ['PMID','doi', 'title', 'authors', 'text']
    read_pmids = df["PMID"].astype(str).tolist()
    missing_elements = []
    # Convert arrays to sets for faster comparison
    set1 = set(arr1)
    set2 = set(read_pmids)

    # Find elements in arr1 that are not in arr2
    missing_elements = list(set1 - set2)
    print(f"{i} : Missing elements: {len(missing_elements)}")
    return missing_elements

In [ ]:
find_missing_elements(43)

In [ ]:
tempDict = {}
for i in range (1,52):
  tempDict[i] = find_missing_elements(i)

In [ ]:
find_missing_elements(43)

In [ ]:
tempDict[2]

In [ ]:
missed_pmids = tempDict[42]

In [ ]:
len(missed_pmids)

In [ ]:
def addMissedPmidsToCSV(i):
  missed_pmids = find_missing_elements(i)
  csv_abstract_filename = f"/content/drive/MyDrive/NLPT(Abstract Data)/data_backup/extracted_data/extracted_text_{i}.csv"
  api_rate_limiter = 0
  for i in range(0, len(missed_pmids)):
    print(i)
    extracted_text = getAbstractData(missed_pmids[i],csv_abstract_filename)
    api_rate_limiter += 1
    # if(api_rate_limiter == 9):
    #   time.sleep(1)
    #   api_rate_limiter = 0

In [ ]:
for id in tempDict:
  if(len(tempDict[id])>0):
    addMissedPmidsToCSV(id)

In [ ]:
addMissedPmidsToCSV(6)